In [1]:
!pip install tifffile

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 706kB 49.6MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
import numpy as np
import pandas as pd
import os
from skimage.transform import resize
import pickle
import matplotlib
import matplotlib.pyplot as plt
from sklearn import cluster

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold

from sklearn.utils import shuffle

from sklearn import preprocessing

In [0]:
############################################
## Functions

def scaleImgPerc(imgIn, percLow, percHigh):
    if len(imgIn.shape) == 2:
        imgIn = imgIn.reshape([imgIn.shape[0], imgIn.shape[1], 1])
        
    w, h, d = imgIn.shape
    imgIn = np.reshape(imgIn, [w * h, d]).astype(np.float64)
    mins = np.percentile(imgIn, percLow, axis=0)
    ranges = np.percentile(imgIn, percHigh, axis=0) - mins
    imgIn = (imgIn - mins[None, :]) / ranges[None, :]
    imgIn = np.reshape(imgIn, [w, h, d])
    imgIn = imgIn.clip(0, 1)
    return imgIn

def imgToFeatVec(imgIn):
    dataMat = pickle.load(open(imgIn, "rb"))
    dataMat = dataMat[:,:,0].squeeze().flatten()
    return dataMat

In [0]:
############################################
## Hard coded paths and vars
BDIR = '/content/drive/My Drive/PrjData/Prj_EuroSat/Out/Data_p20_c2-4-5-8-9'
listImg = BDIR + '/ListImages_p20_c2-4-5-8-9.csv'

SEL_LABEL = 'Forest'

In [0]:
## Read data
df = pd.read_csv(listImg)
df['MatName'] = BDIR + '/ImagesMat/' + df.ImgName + '.p'

df['CAT'] = 0
df.loc[df.CType==SEL_LABEL, 'CAT'] = 1

featVecSize = imgToFeatVec(df.MatName[0]).shape[0]
dataMat = np.zeros([df.shape[0], featVecSize])

for i, tmpRow in df.iterrows():
    dataMat[i,:] = imgToFeatVec(tmpRow.MatName)
labels = np.array(df.CAT)


In [0]:
### Select balanced training data
indPos = np.where(labels==1)[0]
indNeg = shuffle(np.where(labels==0)[0], random_state=42)[0:indPos.shape[0]]

indBal = shuffle(np.concatenate([indPos,indNeg]), random_state=42)

XBal = dataMat[indBal, :]
YBal = labels[indBal]

scaler = preprocessing.StandardScaler().fit(XBal)
XBalSc = scaler.transform(XBal)

X_train, X_test, y_train, y_test = train_test_split(XBalSc, YBal, stratify = YBal, train_size = 0.5, random_state=42)

In [8]:
svc = SVC(kernel='linear')

print('Train size: ' + str(y_train.shape[0]))

### Training the model
print('Training the model ...')
svc.fit(X_train, y_train)

print('Applying the model ...')
yPred = svc.predict(X_test)

Train size: 600
Training the model ...
Applying the model ...


In [9]:
print( 'Accuracy: ' + str(100 * np.sum(yPred==y_test) / y_test.shape[0]))

Accuracy: 84.83333333333333
